# CM20220 Fundamentals of Machine Learning
# Lab 3 Hidden Markov Models (Viterbi Algorithm)


Implement the Viterbi algorithm with backtracking to determine the most likely set of states that would produce a given set of observations.

Given the sequence of observations GGCACTGAA.
Implement the dynamic Viterbi algorithm to compute the missing entries in the table below and then backtrack to identify a most probable path and therefore the most likely sequence of states that would produce the observations.

Do not confuse the Viterbi algorithm with the forward-backward algorithm cover in the lectures. In this task, you will write two loops. The first will calculate the missing values making use of Equation 1. At each step you must also record which decision was taken in the max() calculation. Your second loop will then backtrack using this decision information not the values calculated to determine the most likely path.
Your code should output the values calculated, the decisions made and the most likely sequence (in the right order.)

In [1]:
import pandas as pd
import numpy as np


class HMM:

    def __init__(self, states, observations, start_p, trans_p, emit_p):
        self._states = states
        self._observations = observations
        self._start_p = start_p
        self._trans_p = trans_p
        self._emit_p = emit_p
        
    def _dec_to_state(self, decisions, T, M):
        arr = np.empty((M, T), dtype=str) 
        
        for x in range (T):
            for y in range(M):
                if decisions[y,x] == 0:
                    arr[y][x] = 'L'
                else:
                    arr[y][x] = 'H'
        return arr

    def viterbi(self, sequence):
        T = len(sequence)
        M = len(self._states)
        V = np.zeros((M, T))
        decisions = np.zeros((M, T))

        for i, st in enumerate(self._states):
            V[i][0] = start_p[st] + emit_p[st][sequence[0]]
            decisions[i][0] = i

        for t in range(1, T):

            for s, st in enumerate(self._states):
                max_tr_prob = V[0][t - 1] + self._trans_p[self._states[0]][st]  # L to (L or H)
                tr_prob = V[1][t - 1] + self._trans_p[self._states[1]][st]  # H to (L or H)

                if tr_prob > max_tr_prob:
                    decisions[s][t] = 1
                    max_tr_prob = tr_prob
                else:
                    decisions[s][t] = 0

                max_prob = max_tr_prob + emit_p[st][sequence[t]]
                V[s][t] = max_prob

        opt = np.zeros(T)
        last_state = np.argmax(V[:,-1])
        opt[0] = last_state

        back_t = 1
        for i in range(T - 2, -1, -1):
            opt[back_t] = int(decisions[int(last_state), i])
            last_state = int(decisions[int(last_state), i])
            back_t += 1
        
        arr = self._dec_to_state(decisions, T, M)
        opt = np.flip(opt, axis=0)
        result = []
        for s in opt:
            if s == 0:
                result.append('L')
            else:
                result.append('H')

   
        return pd.DataFrame(V, columns=sequence, index=self._states), result, arr

In [2]:
sequence=("G", "G", "C", "A", "C", "T", "G", "A", "A")
states = ("L", "H")
observations = ("A", "C", "G", "T")
start_p = {"L": -1, "H": -1}
trans_p = {
    "L": {"L": -0.737, "H": -1.322},
    "H": {"L": -1, "H": -1}
    }
emit_p = {
    "L": {"A": -1.737, "C": -2.322, "G": -2.322, "T": -1.737},
    "H": {"A": -2.322, "C": -1.737, "G": -1.737, "T": -2.322}
    }

hmm = HMM(states, observations, start_p, trans_p, emit_p)

## Task 1 (3 marks)
Calculate values and decisions 

In [3]:
V, path, decisions = hmm.viterbi(sequence)
print("Probability Table")
display(V)
print("Decisions")
df1 = pd.DataFrame(decisions, columns=sequence, index=states)
display(df1)

Probability Table


,G,G,C,A,C,T,G,A,A
L,-3.322,-6.059,-8.796,-10.948,-14.007,-16.481,-19.54,-22.014,-24.488
H,-2.737,-5.474,-8.211,-11.533,-14.007,-17.329,-19.54,-22.862,-25.658


Decisions


,G,G,C,A,C,T,G,A,A
L,L,H,H,H,L,L,L,L,L
H,H,H,H,H,L,H,L,H,L


## Task 2 (2 marks)
Calculate the most probable path

In [4]:
df2 = pd.DataFrame(path).T
df2.columns = sequence
df2.index = {'Path'}
print("Path")
display(df2)


Path


,G,G,C,A,C,T,G,A,A
Path,H,H,H,H,L,L,L,L,L


## Marking Guidance

The deadline for all four tasks of this lab sheet is Friday 30th April 2021, 8pm.
You must upload your Jupyter Notebook containing all the tasks attempted to Moodle for this unit by the deadline for this assignment or by any agreed extension deadline. Failing to do so will mean you do not receive the marks for the work. Marks will be given for each task successfully completed. This labsheet gives you an indicate of the last result you should be expecting for task 1. Tasks that are incomplete or produce the wrong answer will receive no marks. An allowance will be made for rounding errors in calculations. You must upload a version of your notebook that includes the output of running the code. We only expect to run your notebooks in exceptional cases.